# PCA
Метод главных компонент (PCA) - это статистический метод уменьшения размерности данных, который позволяет сократить количество переменных, т.е. признаков, в данных. Он основан на нахождении линейно некоррелированных переменных, называемых главными компонентами. PCA очень часто используется в статистике и машинном обучении, поскольку помогает выделить наиболее значимую информацию из всего датасета, который может состоять из сотни признаков [[1](https://spark-school.ru/blogs/pca-spark/)].

Принцип метода поиска аномалий PCA можно описать следующим образом:

1. Данные выражаются в виде ортонормированных векторов, т.е. угол между ними равен 90°. Это реализуется за счет вычисления собственных векторов (eigenvectors).
2. Собственные векторы сортируются в порядке важности, путем рассмотрения вклада каждого в разброс данных в целом.
3. Выбираются только самые важные компоненты, т.е. те, которые объясняют данные более полно, чем остальные. А поскольку они уже сортированы, то выбираются первые.
4. На основе выбранных главных компонент строится новое пространство признаков, в котором каждый объект представлен более компактно, чем в исходном пространстве признаков. Объекты, которые находятся далеко от центра облака точек в новом пространстве, могут быть считаться аномальными.

Таким образом, PCA может использоваться для поиска аномалий в данных. Однако, важно помнить, что PCA является методом уменьшения размерности, и при его использовании может происходить потеря информации. Поэтому, перед применением PCA для поиска аномалий, необходимо тщательно оценить, насколько важна каждая переменная в данных и какие признаки могут быть исключены без потери важной информации.

Работа алгоритма:

1. Мы загружаем набор данных и применяем PCA для уменьшения размерности данных до n измерений.
2. Затем мы реконструируем данные, используя уменьшенную размерность, и вычисляем остатки.
3. Мы задаем пороговое значение для остатков, чтобы отличить нормальные и аномальные точки данных.
4. Определяем аномальные точки данных, у которых остатки превышают пороговое значение.

In [ ]:
# ! git clone -b ml_and_stat_methods https://github.com/mipt-nd/novelty-detection.git
# %cd novelty-detection/

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')
# ! cp -r /content/drive/MyDrive/Study/MIPT_magistery/qualification_work/data .

In [1]:
import os
import random
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA

# metrics
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

def seed_everything(seed):
    # фискирует максимум сидов для корректности сравнения разных экспериментов
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed(seed)
    # torch.backends.cudnn.deterministic = True
SEED = 42
seed_everything(SEED)

# Ignore all warnings
import warnings
warnings.filterwarnings('ignore')

# Datasets

In [2]:
# Синтетически сгенерированные нормально распределенные данные
def make_norm_data(rows, columns, noise_percent=0.05):
    data_norm = pd.DataFrame(data=np.random.normal(
                            loc=0, scale=1, size=(rows, columns)))
    noise = pd.DataFrame(data=np.random.uniform(
                            low=-6, high=6, size=(int(data_norm.shape[0]*noise_percent), data_norm.shape[1])))

    data_noise = pd.DataFrame()
    for feature in noise.columns:
        filter = (
            (noise[feature] < data_norm[feature].min())
            | (noise[feature] > data_norm[feature].max())
            )
        data_noise = pd.concat([data_noise, noise[filter]])
    data_noise = data_noise.drop_duplicates()
    data_norm['anomaly'] = 0
    data_noise['anomaly'] = 1

    df_norm = pd.concat((data_norm, data_noise))
    return df_norm

df_norm = make_norm_data(1000, 8)
df_norm

,0,1,2,3,4,5,6,7,anomaly
0,0.496714,-0.138264,0.647689,1.523030,-0.234153,-0.234137,1.579213,0.767435,0
1,-0.469474,0.542560,-0.463418,-0.465730,0.241962,-1.913280,-1.724918,-0.562288,0
2,-1.012831,0.314247,-0.908024,-1.412304,1.465649,-0.225776,0.067528,-1.424748,0
3,-0.544383,0.110923,-1.150994,0.375698,-0.600639,-0.291694,-0.601707,1.852278,0
4,-0.013497,-1.057711,0.822545,-1.220844,0.208864,-1.959670,-1.328186,0.196861,0
...,...,...,...,...,...,...,...,...,...
32,0.063984,1.772509,0.031678,-0.098264,-3.938531,-2.655435,-4.093768,-3.652004,1
38,-2.800126,2.593122,-1.268901,2.124984,-0.288220,5.925277,2.746826,5.429896,1
5,2.151800,-0.353991,-1.345218,-1.488481,1.680586,-0.062468,4.189587,4.496424,1
37,-2.554173,0.008819,-0.528646,-1.866863,2.324896,3.114742,4.235808,5.674279,1


In [3]:
# SKAB data

all_files=[]
for root, dirs, files in os.walk("data_labeled/SKAB/"):
    for file in files:
        if file.endswith(".csv"):
             all_files.append(os.path.join(root, file))

# формируем датафрейм
dfs=[]
for path in all_files:
    df = pd.read_csv(path,index_col='datetime',sep=';',parse_dates=True)
    # print(path, df.shape)
    dfs.append(df)
dfs = [df for df in dfs if df.shape[1] == 10]
df_skab = pd.concat(dfs)
df_skab = df_skab.drop_duplicates()
df_skab = df_skab.drop('changepoint', axis=1).sort_index()
display(df_skab)

,Accelerometer1RMS,Accelerometer2RMS,Current,Pressure,Temperature,Thermocouple,Voltage,Volume Flow RateRMS,anomaly
datetime,,,,,,,,,
2020-02-08 16:06:48,0.213628,0.266664,2.588900,-0.273216,89.1732,29.3477,231.257,125.3240,0.0
2020-02-08 16:06:49,0.214988,0.267634,2.994500,0.382638,89.3237,29.3409,234.948,125.6780,0.0
2020-02-08 16:06:50,0.212976,0.268121,2.849810,0.054711,88.9180,29.3450,208.112,125.3240,0.0
2020-02-08 16:06:51,0.214286,0.267545,2.884770,-0.273216,89.0148,29.3459,206.579,125.6780,0.0
2020-02-08 16:06:52,0.213402,0.266691,2.595010,0.382638,88.9571,29.3462,223.472,125.3240,0.0
...,...,...,...,...,...,...,...,...,...
2020-03-09 17:14:04,0.026853,0.038926,0.740614,0.054711,69.6371,24.1045,237.276,32.0451,0.0
2020-03-09 17:14:05,0.027067,0.038430,0.988875,0.054711,69.6731,24.1046,230.729,32.9562,0.0
2020-03-09 17:14:07,0.027582,0.038836,0.588439,0.054711,69.6959,24.1020,233.443,32.0000,0.0


In [5]:
# labeled data (https://github.com/xuhongzuo/deep-iforest)

df_pageblocks = pd.read_csv('data_labeled/pageblocks_16.csv')
df_pageblocks = df_pageblocks.rename(columns={'label': 'anomaly'})
df_shuttle = pd.read_csv('data_labeled/shuttle_16.csv')
df_shuttle = df_shuttle.rename(columns={'label': 'anomaly'})
print('df_pageblocks')
display(df_pageblocks.head())
print('df_shuttle')
display(df_shuttle.head())

df_pageblocks


,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,anomaly
0,0.004981,0.010870,0.000194,0.002594,0.367089,0.634328,0.000268,0.000212,0.000347,0.001557,0
1,0.006227,0.010870,0.000243,0.002160,0.397679,0.873134,0.000525,0.000333,0.000650,0.001246,0
2,0.006227,0.030797,0.000701,0.005574,0.247890,0.723881,0.000692,0.000727,0.001583,0.001869,0
3,0.004981,0.010870,0.000194,0.002594,0.336498,0.726013,0.000672,0.000182,0.000412,0.000623,0
4,0.006227,0.003623,0.000076,0.000918,0.472574,0.940299,0.000252,0.000061,0.000217,0.000934,0


df_shuttle


,att1,att2,att3,att4,att5,att6,att7,att8,att9,anomaly
0,0.214286,0.607955,0.12766,0.020145,0.207865,0.088428,0.347222,0.746875,0.737113,1
1,0.085714,0.607955,0.12766,0.020145,0.207865,0.088428,0.458333,0.746875,0.716495,1
2,0.200000,0.607955,0.12766,0.020145,0.067416,0.088428,0.361111,0.912500,0.871134,1
3,0.085714,0.602273,0.12766,0.020145,0.207865,0.088428,0.472222,0.746875,0.716495,1
4,0.185714,0.653409,0.12766,0.020145,0.067416,0.088428,0.375000,0.912500,0.865979,1


In [6]:
# для автоматизации формируем словарь из наборов данных 

datasets = {
    'df_norm': df_norm,
    'df_skab': df_skab,
    'df_pageblocks': df_pageblocks,
    'df_shuttle': df_shuttle
}

# Метрики

In [7]:
# воспользуемся методом classification_report из sklearn

def score_metrics(real_outliers, pred_outliers, output_dict=False):
    scores = {}
    # scores['f1_score'] = f1_score(real_outliers, pred_outliers)
    scores = classification_report(real_outliers, pred_outliers, output_dict=output_dict)
    return scores

# Preprocessing

In [13]:
def preprocessing(df, scaler):
    df = df.drop_duplicates()
    X = df.copy()
    y = X.pop('anomaly')

    # preprocessing
    columns = list(X.columns)
    X = scaler.fit_transform(X)
    X = pd.DataFrame(data=X, columns=columns)
    return X, y

# PCA

В этом примере мы загружаем набор данных и применяем PCA для уменьшения размерности данных до n измерений (автоподбор по методу [Minka's MLE](https://vismod.media.mit.edu/tech-reports/TR-514.pdf)). Затем мы реконструируем данные, используя уменьшенную размерность, и вычисляем остатки. Затем задается пороговое значение для остатков, чтобы отличить нормальные и аномальные точки данных. Точки данных, у которых остатки превышают пороговое значение - аномальные.

In [17]:
scalers = {'std scaler': StandardScaler(),
          'MinMax scaler': MinMaxScaler()}

for scaler_name, scaler in scalers.items():
    print(scaler_name)
    for df_name, df in datasets.items():
        print(df_name)
        X, y = preprocessing(df, scaler)

        # Применим PCA для снижения размерности данных
        pca = PCA(n_components='mle')
        '''
        If ``n_components == 'mle'`` and ``svd_solver == 'full'``, Minka's
        MLE is used to guess the dimension. Use of ``n_components == 'mle'``
        will interpret ``svd_solver == 'auto'`` as ``svd_solver == 'full'``
        '''
        X_pca = pca.fit_transform(X)

        # Реконструируем данные, используя уменьшенную размерность
        X_reconstructed = pca.inverse_transform(X_pca)

        # Вычислим остатки
        residuals = np.linalg.norm(X - X_reconstructed, axis=1)

        # Укажем пороговое значение для остатков, чтобы различать нормальные и аномальные точки данных
        threshold = np.percentile(residuals, 95)

        # Определим аномальные точки данных, у которых остатки превышают пороговое значение
        anomalies = X[residuals > threshold]
        anomalies['anomaly'] = 1
        X['anomaly'] = anomalies['anomaly']
        X['anomaly'] = X['anomaly'].fillna(0)

        print(score_metrics(y, X['anomaly']))

        print('-' * 10)

std scaler
df_norm
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1000
           1       0.94      1.00      0.97        50

    accuracy                           1.00      1050
   macro avg       0.97      1.00      0.98      1050
weighted avg       1.00      1.00      1.00      1050

----------
df_skab
              precision    recall  f1-score   support

         0.0       0.67      0.99      0.80     23851
         1.0       0.82      0.12      0.20     13067

    accuracy                           0.68     36918
   macro avg       0.75      0.55      0.50     36918
weighted avg       0.72      0.68      0.59     36918

----------
df_pageblocks
              precision    recall  f1-score   support

           0       0.91      0.95      0.93      4883
           1       0.18      0.10      0.13       510

    accuracy                           0.87      5393
   macro avg       0.55      0.53      0.53      5393
weighted av